### Models built for check-in in next month prediction (without the imbalanced dataset handling, but use class weight adjustment in the models)
### Training dataset is up to end of Jun.2024 to see how models predict Jul.2024 check-in
### The area under the precision-recall curve is around 5.6 which reflects the high recall, low precision nature for class 1 of previous notebooks

In [3]:
### read in the necessary files
import pandas as pd
usr_segment = pd.read_csv("Jun_segmentation.csv")
features = pd.read_csv("features.csv")
openapp_recency = pd.read_csv("openapp_recency.csv")

In [4]:
merged_1 = pd.merge(usr_segment, features, on='user_sn', how='inner')
merged_2 = pd.merge(merged_1, openapp_recency, on='user_sn', how='inner')

In [3]:
merged_2.dtypes

user_sn                        int64
segmentation                   int64
hourly_ci_number               int64
ovn_ci_number                  int64
day_ci_number                  int64
total_paid_from_user           int64
usr_cancel_num_3_months        int64
hotel_cancel_num_3_months      int64
g2j_cancel_num_3_months        int64
no_show_num_3_months           int64
recency                        int64
average_ci_time_gap          float64
std_ci_time_gap              float64
open_app_num_3_months        float64
mileage_used_num             float64
current_mileage_point        float64
search_num_3_months          float64
review_num_3_months          float64
avg_mark                     float64
time_since_join                int64
user_province                 object
ci_num                         int64
openapp_recency                int64
dtype: object

In [4]:
merged_2.isnull().sum()

user_sn                           0
segmentation                      0
hourly_ci_number                  0
ovn_ci_number                     0
day_ci_number                     0
total_paid_from_user              0
usr_cancel_num_3_months           0
hotel_cancel_num_3_months         0
g2j_cancel_num_3_months           0
no_show_num_3_months              0
recency                           0
average_ci_time_gap               0
std_ci_time_gap                   0
open_app_num_3_months        117067
mileage_used_num                239
current_mileage_point         50134
search_num_3_months          156746
review_num_3_months          103449
avg_mark                     103449
time_since_join                   0
user_province                     2
ci_num                            0
openapp_recency                   0
dtype: int64

### feature explanation
- segmentation: user segmentation by the month of train period

- hourly_ci_number: user's number of hourly check-ins

- ovn_ci_number: user's number of ovn check-ins 

- day_ci_number: user's number of day check-ins 

- total_paid_from_user: user's paid amount 

- usr_cancel_num_3_months: number of user cancels in the last 3 months in train period month

- hotel_cancel_num_3_months: number of user cancels in the last 3 months in train period month

- g2j_cancel_num_3_months: number of user cancels in the last 3 months  in train period month

- no_show_num_3_months: number of noshow in the last 3 months in train period month

- recency: the day difference between last check-in date before end of train period month and end of train period month

- average_ci_time_gap: average day gap between check-ins

- std_ci_time_gap: standard deviation of day gaps between check-ins

- open_app_num_3_months: number of times user open app in the last 3 months in the train period month

- mileage_used_num: number of times users used mileage points until end of train period month

- current_mileage_point: mileage points of users by the end of train period month

- search_num_3_months: number of times user search in the last 3 months in the train period month

- review_num_3_months: number of times user give review in the last 3 months in the train period month

- time_since_join: day gap between user's register date and end of train period month

- user_province: Province of user

- openapp_recency: the day difference between last open-app date before end of train period month and end of train period month

- ci_num: number of user's check-ins in next month period (used to create target variable)

## Cleaning data

In [5]:
## Fill the null values with 0 for open_app_num_3_months, mileage_used_num, current_mileage_point
## search_num_3_months, review_num_3_months, avg_mark
merged_2[['open_app_num_3_months', 'mileage_used_num', 'current_mileage_point', 
          'review_num_3_months','search_num_3_months']] = merged_2[['open_app_num_3_months', 'mileage_used_num', 'current_mileage_point',
                                                            'review_num_3_months','search_num_3_months',]].fillna(0).astype('int64')

merged_2['avg_mark'] = merged_2['avg_mark'].fillna(0).astype('float32')

In [6]:
# Get rid of records with time_since_join or average_ci_time_gap < 0 and records with user_province as null
merged_2 = merged_2[merged_2['time_since_join']>=0]
merged_2 = merged_2[merged_2['average_ci_time_gap']>=0]
merged_2 = merged_2[merged_2['user_province'].isnull()==False]

In [7]:
segment_data = {
    'segmentation': [1, 2, 3, 4, 5, 6, 7],
    'segmentation_text': ['New', 'Existing', 'Retention', 'Win-back', 'Churn', 'Drop', 'Dormant']
}

segment_df = pd.DataFrame(segment_data)

# add the segementaion test
cleaned_df = pd.merge(merged_2, segment_df, on='segmentation', how='left')
cleaned_df.shape

(265085, 24)

### Create target/dependent variable and train test split

In [8]:
cleaned_df['ci_or_not'] = cleaned_df['ci_num'].apply(lambda x: 1 if x > 0 else 0)
cleaned_df['ci_or_not'].value_counts()

ci_or_not
0    242262
1     22823
Name: count, dtype: int64

In [51]:
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import classification_report

# X = cleaned_df.drop(columns=['user_sn','segmentation', 'ci_or_not', 'ci_num'],axis=1)
# y = cleaned_df['ci_or_not']
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=32, stratify=y)

In [10]:
X = cleaned_df.drop(columns=['user_sn','segmentation', 'ci_or_not', 'ci_num'],axis=1)
y = cleaned_df['ci_or_not']

In [11]:
num_features = X.select_dtypes(exclude="object").columns
cat_features = X.select_dtypes(include="object").columns

print('We have {} numerical features : {}'.format(len(num_features), num_features))
print('\nWe have {} categorical features : {}'.format(len(cat_features), cat_features))

We have 19 numerical features : Index(['hourly_ci_number', 'ovn_ci_number', 'day_ci_number',
       'total_paid_from_user', 'usr_cancel_num_3_months',
       'hotel_cancel_num_3_months', 'g2j_cancel_num_3_months',
       'no_show_num_3_months', 'recency', 'average_ci_time_gap',
       'std_ci_time_gap', 'open_app_num_3_months', 'mileage_used_num',
       'current_mileage_point', 'search_num_3_months', 'review_num_3_months',
       'avg_mark', 'time_since_join', 'openapp_recency'],
      dtype='object')

We have 2 categorical features : Index(['user_province', 'segmentation_text'], dtype='object')


### Preprocessing and train for Tree-based models

In [13]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
oh_transformer = OneHotEncoder(sparse_output=False)
numeric_transformer = StandardScaler()

preprocessor = ColumnTransformer(
    [
        ("OneHotEncoder", oh_transformer, cat_features),    
        ('pass', 'passthrough', num_features)
    ]
)

In [14]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold
from sklearn.metrics import make_scorer, precision_recall_curve, auc


stratified_kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

# Define a custom PRC AUC scoring function
def prc_auc_score(y_true, y_scores):
    precision, recall, _ = precision_recall_curve(y_true, y_scores)
    return auc(recall, precision)

# Create a custom scorer using make_scorer
prc_auc_scorer = make_scorer(prc_auc_score, response_method='predict_proba')

# Define the model and parameter grid
rf_pipeline = Pipeline([
    ('preprocessor', preprocessor),  # Apply the preprocessing steps
    ('classifier', RandomForestClassifier())  # Fit the model
])

param_dist = {
    'classifier__n_estimators': [50, 100, 150, 200],
    'classifier__max_features': ['auto', 'sqrt', 'log2'],
    'classifier__max_depth': [None, 10, 20, 30, 40, 50],
    'classifier__min_samples_split': [2, 5, 10],
    'classifier__min_samples_leaf': [1, 2, 4],
    'classifier__bootstrap': [True, False],
    'classifier__class_weight': [{0: 1, 1: 5}, {0: 1, 1: 10}]
}

# Setup GridSearchCV with the custom scorer
random_search = RandomizedSearchCV(
    estimator=rf_pipeline,
    param_distributions=param_dist,
    n_iter=3,
    scoring=prc_auc_scorer,
    cv=stratified_kfold,
    verbose=3,
    random_state=42,
    n_jobs=2
)

# Fit the model
random_search.fit(X, y)

# Get the best parameters and score
best_params = random_search.best_params_
best_score = random_search.best_score_

print(f"Best parameters: {best_params}")
print(f"Best PRC AUC score: {best_score}")

Fitting 3 folds for each of 3 candidates, totalling 9 fits
[CV 2/3] END classifier__bootstrap=True, classifier__class_weight={0: 1, 1: 10}, classifier__max_depth=10, classifier__max_features=log2, classifier__min_samples_leaf=4, classifier__min_samples_split=10, classifier__n_estimators=50;, score=0.547 total time=   4.6s
[CV 1/3] END classifier__bootstrap=True, classifier__class_weight={0: 1, 1: 10}, classifier__max_depth=10, classifier__max_features=log2, classifier__min_samples_leaf=4, classifier__min_samples_split=10, classifier__n_estimators=50;, score=0.553 total time=   4.7s
[CV 3/3] END classifier__bootstrap=True, classifier__class_weight={0: 1, 1: 10}, classifier__max_depth=10, classifier__max_features=log2, classifier__min_samples_leaf=4, classifier__min_samples_split=10, classifier__n_estimators=50;, score=0.554 total time=   4.6s
[CV 1/3] END classifier__bootstrap=True, classifier__class_weight={0: 1, 1: 10}, classifier__max_depth=50, classifier__max_features=log2, classifi

/Users/trihoang/Desktop/g2j_project/venv/lib/python3.11/site-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Best parameters: {'classifier__n_estimators': 150, 'classifier__min_samples_split': 2, 'classifier__min_samples_leaf': 2, 'classifier__max_features': 'sqrt', 'classifier__max_depth': 40, 'classifier__class_weight': {0: 1, 1: 10}, 'classifier__bootstrap': True}
Best PRC AUC score: 0.5705624975013207


In [16]:
from imblearn.ensemble import EasyEnsembleClassifier
from sklearn.metrics import classification_report

stratified_kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

# Define a custom PRC AUC scoring function
def prc_auc_score(y_true, y_scores):
    precision, recall, _ = precision_recall_curve(y_true, y_scores)
    return auc(recall, precision)

# Create a custom scorer using make_scorer
prc_auc_scorer = make_scorer(prc_auc_score, response_method='predict_proba')


param_dist = {
    'classifier__n_estimators': [50, 100, 150, 200],  # Number of base estimators in the ensemble
    'classifier__sampling_strategy': ['auto', 'all'],  # Strategy for sampling
    'classifier__replacement': [True],  # Whether sampling should be with replacement
    'classifier__random_state': [42]  # Seed for random number generator
}

rf_pipeline = Pipeline([
    ('preprocessor', preprocessor),  # Apply the preprocessing steps
    ('classifier', EasyEnsembleClassifier())  # Fit the model
])


random_search = RandomizedSearchCV(
    estimator=rf_pipeline,
    param_distributions=param_dist,
    n_iter=3,
    scoring=prc_auc_scorer,
    cv=stratified_kfold,
    verbose=3,
    random_state=42,
    n_jobs=2
)

# Fit the model
random_search.fit(X, y)

# Get the best parameters and score
best_params = random_search.best_params_
best_score = random_search.best_score_

print(f"Best parameters: {best_params}")
print(f"Best PRC AUC score: {best_score}")

Fitting 3 folds for each of 3 candidates, totalling 9 fits
[CV 1/3] END classifier__n_estimators=50, classifier__random_state=42, classifier__replacement=True, classifier__sampling_strategy=all;, score=0.562 total time= 1.7min
[CV 2/3] END classifier__n_estimators=50, classifier__random_state=42, classifier__replacement=True, classifier__sampling_strategy=all;, score=0.555 total time= 1.7min
[CV 3/3] END classifier__n_estimators=50, classifier__random_state=42, classifier__replacement=True, classifier__sampling_strategy=all;, score=0.562 total time= 1.7min
[CV 1/3] END classifier__n_estimators=150, classifier__random_state=42, classifier__replacement=True, classifier__sampling_strategy=all;, score=0.562 total time= 5.2min
[CV 2/3] END classifier__n_estimators=150, classifier__random_state=42, classifier__replacement=True, classifier__sampling_strategy=all;, score=0.557 total time= 5.2min
[CV 1/3] END classifier__n_estimators=50, classifier__random_state=42, classifier__replacement=True

### Preprocessing and train for non-tree-based models

In [12]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

oh_transformer = OneHotEncoder()
numeric_transformer = StandardScaler()


preprocessor = ColumnTransformer(
    [
        ("OneHotEncoder", oh_transformer, cat_features),
         ("StandardScaler", numeric_transformer, num_features),        
    ]
)

In [21]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.metrics import precision_recall_curve, auc

class_weight = {0: 1.0, 1: 5.0} 

# Define the ANN model architecture
def build_ann():
    model = Sequential()
    model.add(Input(shape=(89,)))  # Specify the input shape using Input layer
    model.add(Dense(64, activation='relu', kernel_initializer='glorot_uniform'))
    model.add(Dropout(0.3))
    model.add(Dense(32, activation='relu', kernel_initializer='glorot_uniform'))
    model.add(Dropout(0.3))
    model.add(Dense(1, activation='sigmoid', kernel_initializer='glorot_uniform'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Wrap the Keras model with KerasClassifier
ann_model = KerasClassifier(model=build_ann, verbose=0)

# Create the pipeline with preprocessing and the Keras model
Ann_pipeline = Pipeline([
    ('preprocessor', preprocessor),  # Apply the preprocessing steps
    ('classifier', ann_model)  # Fit the model
])

# Initialize StratifiedKFold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=32)

# Store scores for each fold
scores = []

for train_index, test_index in skf.split(X, y):
    # Split data
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Train the model
    Ann_pipeline.fit(X_train, y_train, classifier__class_weight=class_weight)

    # Predict on the test set
    y_pred_proba = Ann_pipeline.predict_proba(X_test)[:, 1]

    # Evaluate the model
    precision, recall, _ = precision_recall_curve(y_test, y_pred_proba)
    auc_score = auc(recall, precision)
    scores.append(auc_score)
    print(f"Fold PRC AUC score: {auc_score:.4f}")

# Print the average accuracy across all folds
print(f"PRC AUC score: {np.mean(scores):.4f}")

Fold PRC AUC score: 0.5477
Fold PRC AUC score: 0.5389
Fold PRC AUC score: 0.5389
Fold PRC AUC score: 0.5344
Fold PRC AUC score: 0.5338
PRC AUC score: 0.5388
